In [ ]:
# !pip install transformers
# !pip install datasets

In [1]:
import pandas as pd
import torch
import datasets
import transformers
from transformers import AutoTokenizer,AutoModelForMaskedLM
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments
from transformers import Trainer
import warnings
warnings.filterwarnings("ignore")

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [3]:
train_df=pd.read_csv('EmbibleDataset/train_df.csv', sep='\\t', encoding='utf-8')
valid_df=pd.read_csv('EmbibleDataset/valid_df.csv', sep='\\t', encoding='utf-8')

In [4]:
train_df=train_df.drop(columns=["verse_idx.1","verse.1","name"])
#train_df["name.1"].rename("name")
train_df.rename(columns={"name.1": 'name'}, inplace=True)
#valid_df=train_df.drop(columns=["verse_idx.1","verse.1","name"])
#train_df["name.1"].rename("name")
#valid_df.rename(columns={"name.1": 'name'}, inplace=True)

In [5]:
print(train_df)
print(valid_df)

      verse_idx   verse                                               name
0        Isaiah     0.0  חֲזוֹן֙ יְשַֽׁעְיָ֣הוּ בֶן אָמ֔וֹץ אֲשֶׁ֣ר חָז...
1        Isaiah     1.0  שִׁמְע֤וּ שָׁמַ֙יִם֙ וְהַאֲזִ֣ינִי אֶ֔רֶץ כִּ֥...
2        Isaiah     2.0  יָדַ֥ע שׁוֹר֙ קֹנֵ֔הוּ וַחֲמ֖וֹר אֵב֣וּס בְּעָ...
3        Isaiah     3.0  ה֣וֹי גּ֣וֹי חֹטֵ֗א עַ֚ם כֶּ֣בֶד עָוֺ֔ן זֶ֣רַע...
4        Isaiah     4.0  עַ֣ל מֶ֥ה תֻכּ֛וּ ע֖וֹד תּוֹסִ֣יפוּ סָרָ֑ה כָּ...
...         ...     ...                                                ...
22145    Exodus  1208.0  וַיְכַ֥ס הֶעָנָ֖ן אֶת אֹ֣הֶל מוֹעֵ֑ד וּכְב֣וֹד...
22146    Exodus  1209.0  וְלֹא יָכֹ֣ל מֹשֶׁ֗ה לָבוֹא֙ אֶל אֹ֣הֶל מוֹעֵ֔...
22147    Exodus  1210.0  וּבְהֵעָל֤וֹת הֶֽעָנָן֙ מֵעַ֣ל הַמִּשְׁכָּ֔ן י...
22148    Exodus  1211.0  וְאִם לֹ֥א יֵעָלֶ֖ה הֶעָנָ֑ן וְלֹ֣א יִסְע֔וּ ע...
22149    Exodus  1212.0  כִּי֩ עֲנַ֨ן יְהוָ֤ה עַֽל הַמִּשְׁכָּן֙ יוֹמָ֔...

[22150 rows x 3 columns]
         name  verse_idx                                              vers

In [6]:
cp = 'onlplab/alephbert-base'

model=AutoModelForMaskedLM.from_pretrained(cp)
tokenizer=AutoTokenizer.from_pretrained(cp)

In [7]:
train_ds=datasets.Dataset.from_pandas(train_df)
valid_ds=datasets.Dataset.from_pandas(valid_df)

In [8]:

def tokenize_function(dataset):
  return tokenizer(str(dataset["verse"]))
tokenized_train_ds=train_ds.map(tokenize_function)
tokenized_valid_ds=valid_ds.map(tokenize_function)


  0%|          | 0/22150 [00:00<?, ?ex/s]

  0%|          | 0/536 [00:00<?, ?ex/s]

In [9]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [10]:
batch_size = 32
# Show the training loss with every epoch
logging_steps = len(tokenized_train_ds)

training_args = TrainingArguments(
    output_dir="EmbibleModels/Embibert-finetuned-AlephBertGimmel-10epochs",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=False,
    logging_steps=logging_steps,
    num_train_epochs=10

)


In [11]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [12]:
import gc

gc.collect()

torch.cuda.empty_cache()
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_valid_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [13]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: verse_idx, name, verse. If verse_idx, name, verse are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 22150
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 6930
  Number of trainable parameters = 126030112
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,1.734000,6.200029
2,1.707500,7.757162
3,1.699100,10.060718
4,1.675400,10.159719
5,1.658800,10.874681
6,1.666500,12.445745
7,1.630400,12.339250
8,1.659600,13.200126
9,1.643900,12.449395
10,1.663800,12.418348


Saving model checkpoint to EmbibleModels/Embibert-finetuned-AlephBertGimmel-10epochs/checkpoint-500
Configuration saved in EmbibleModels/Embibert-finetuned-AlephBertGimmel-10epochs/checkpoint-500/config.json
Model weights saved in EmbibleModels/Embibert-finetuned-AlephBertGimmel-10epochs/checkpoint-500/pytorch_model.bin
tokenizer config file saved in EmbibleModels/Embibert-finetuned-AlephBertGimmel-10epochs/checkpoint-500/tokenizer_config.json
Special tokens file saved in EmbibleModels/Embibert-finetuned-AlephBertGimmel-10epochs/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: __index_level_0__, verse_idx, name, verse. If __index_level_0__, verse_idx, name, verse are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 536
  Batch size = 32
Saving model checkpoint to EmbibleModels/Embibert-fi

TrainOutput(global_step=6930, training_loss=1.6739039565577651, metrics={'train_runtime': 900.3576, 'train_samples_per_second': 246.013, 'train_steps_per_second': 7.697, 'total_flos': 683372820672000.0, 'train_loss': 1.6739039565577651, 'epoch': 10.0})

In [16]:
trainer.save_model("EmbibleModels/Embibert-finetuned-AlephBertGimmel-10epochs")
model.save_pretrained("EmbibleModels/Embibert-finetuned-AlephBertGimmel-10epochs")

Saving model checkpoint to EmbibleModels/Embibert-finetuned-AlephBertGimmel-10epochs
Configuration saved in EmbibleModels/Embibert-finetuned-AlephBertGimmel-10epochs/config.json
Model weights saved in EmbibleModels/Embibert-finetuned-AlephBertGimmel-10epochs/pytorch_model.bin
tokenizer config file saved in EmbibleModels/Embibert-finetuned-AlephBertGimmel-10epochs/tokenizer_config.json
Special tokens file saved in EmbibleModels/Embibert-finetuned-AlephBertGimmel-10epochs/special_tokens_map.json
Configuration saved in EmbibleModels/Embibert-finetuned-AlephBertGimmel-10epochs/config.json
Model weights saved in EmbibleModels/Embibert-finetuned-AlephBertGimmel-10epochs/pytorch_model.bin


In [18]:
cp = 'onlplab/alephbert-base'
model=AutoModelForMaskedLM.from_pretrained(cp)
tokenizer=AutoTokenizer.from_pretrained(cp)

loading configuration file config.json from cache at /home/fono/.cache/huggingface/hub/models--onlplab--alephbert-base/snapshots/1745fb3ff5137e41e9eb4d6246e0758f63b93e46/config.json
Model config BertConfig {
  "_name_or_path": "onlplab/alephbert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 52000
}

loading weights file pytorch_model.bin from cache at /home/fono/.cache/huggingface/hub/models--onlplab--alephbert-base/snapshots/1745fb3ff5137e41e9eb4d6

In [19]:
batch_size = 32
# Show the training loss with every epoch
logging_steps = len(tokenized_train_ds)

training_args = TrainingArguments(
    output_dir="EmbibleModels/Embibert-finetuned-AlephBertGimmel-20epochs",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=False,
    logging_steps=logging_steps,
    num_train_epochs=20

)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [20]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [ ]:
import gc

gc.collect()

torch.cuda.empty_cache()
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_valid_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
)
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: verse_idx, name, verse. If verse_idx, name, verse are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 22150
  Num Epochs = 20
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 13860
  Number of trainable parameters = 126030112


The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


Epoch,Training Loss,Validation Loss
1,1.663200,16.870060
2,1.677900,18.831984
3,1.675400,19.834837
4,1.661600,16.043135
5,1.658300,17.664694


Saving model checkpoint to EmbibleModels/Embibert-finetuned-AlephBertGimmel-20epochs/checkpoint-500
Configuration saved in EmbibleModels/Embibert-finetuned-AlephBertGimmel-20epochs/checkpoint-500/config.json
Model weights saved in EmbibleModels/Embibert-finetuned-AlephBertGimmel-20epochs/checkpoint-500/pytorch_model.bin
tokenizer config file saved in EmbibleModels/Embibert-finetuned-AlephBertGimmel-20epochs/checkpoint-500/tokenizer_config.json
Special tokens file saved in EmbibleModels/Embibert-finetuned-AlephBertGimmel-20epochs/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: __index_level_0__, verse_idx, name, verse. If __index_level_0__, verse_idx, name, verse are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 536
  Batch size = 32
Saving model checkpoint to EmbibleModels/Embibert-fi

In [ ]:
trainer.save_model("EmbibleModels/Embibert-finetuned-AlephBertGimmel-20epochs")
model.save_pretrained("EmbibleModels/Embibert-finetuned-AlephBertGimmel-20epochs")

In [ ]:
cp = 'onlplab/alephbert-base'
model=AutoModelForMaskedLM.from_pretrained(cp)
tokenizer=AutoTokenizer.from_pretrained(cp)

In [ ]:
batch_size = 32
# Show the training loss with every epoch
logging_steps = len(tokenized_train_ds)

training_args = TrainingArguments(
    output_dir="EmbibleModels/Embibert-finetuned-AlephBertGimmel-50epochs",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=False,
    logging_steps=logging_steps,
    num_train_epochs=50
)


In [ ]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [ ]:
import gc

gc.collect()

torch.cuda.empty_cache()
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_valid_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
)
trainer.train()

In [ ]:
trainer.save_model("EmbibleModels/Embibert-finetuned-AlephBertGimmel-50epochs")
model.save_pretrained("EmbibleModels/Embibert-finetuned-AlephBertGimmel-50epochs")